分类树的构造，通过递归的二分法，不断将node分成左右两个子节点。每个子集中，以贪婪方式搜索最重要的特征组合，及其值作为最佳分裂点。分的子集的好坏，以子集中label的纯度度量，指标为Gini不纯度和信息增益。

回归树的生成差不多，只有2点不同：

    1.分裂点的品质度量采用两个子集的MSE；一个子集的MSE等于所有targets (即y值)的方差，加权MSE越小，分裂得越好。
    2.叶节点的值为 一个终节点的target平均值，而分类中是占多数的labels(1比0多，则label为1）。

## 涉及的Python先验：

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import numpy as np

In [3]:
a,b = [1,2]
a
b

1

2

In [4]:
a,b=1

TypeError: 'int' object is not iterable

In [5]:
# 布尔值检索
names = np.array(['Bob', 'Joe', 'Will', 'Bob', 'Will', 'Joe', 'Joe'])
mask = names == 'Bob'
names[mask]  # 选出”Bob“
names[~mask] # 选出非‘Bob‘

array(['Bob', 'Bob'],
      dtype='<U4')

array(['Joe', 'Will', 'Will', 'Joe', 'Joe'],
      dtype='<U4')

## 例：房价预估

定义MSE：

In [6]:
def mse(targets):
    """targets (np.ndarray)
    """
    if targets.size == 0: # 当数据集为空
        return 0
    return np.var(targets)

定义 加权MSE：

In [7]:
def weighted_mse(groups):
    """ Calculate weighted MSE of children after a split
    Arg:
        groups (list of children, and a child consists a list of targets)
    Returns:
        float, weighted impurity
    """
    total = sum(len(group) for group in groups)
    weighted_sum = 0.0
    for group in groups:
        weighted_sum += len(group) / float(total) * mse(group)
    return weighted_sum

In [8]:
mse(np.array([1, 2, 3]))

0.66666666666666663

In [9]:
weighted_mse([np.array([1, 2, 3]), np.array([1, 2])])

0.5

![house price](house_price.png)

穷尽所有（特征, 特征值）pairs，并计算对应的MSE：

In [7]:
# MSE(type, Semi)
weighted_mse([np.array([600, 400, 700]), np.array([700, 800])])
# MSE(bedrooms, 2)
weighted_mse([np.array([700, 400]),np.array([600, 800, 700])])
# MSE(bedrooms, 3)
weighted_mse([np.array([600, 800]),np.array([700, 400, 700])])
# MSE(bedrooms, 4)
weighted_mse([np.array([700]),np.array([600,700,800,400])])

10333.333333333334

13000.0

16000.0

17500.0

MSE(type,Semi)最小，所以选（type, Semi）为root进行分裂
![](type_semi.png)

若深度为1的回归树就够了，即可停止分裂，将左右子集作为叶节点，targets平均值作为叶节点的值。

或者，以右子集进一步生成第二层：

In [8]:
# MSE(bedrooms, 2)
weighted_mse([np.array([400]),np.array([600, 700])])
# MSE(bedrooms, 3)
weighted_mse([np.array([600]),np.array([400, 700])])
# MSE(bedrooms, 4)
weighted_mse([np.array([700]),np.array([600, 400])])

1666.6666666666665

15000.0

6666.6666666666661

![](second_level.png)

# 代码实现：
1. 给定数据集X,y
2. get_best_split(X,y) 的得到root node及左、右2个子集（child node）
    * 其中涉及的操作有split_node，即分裂动作
    * 度量最佳：weighted_mse
3. 确认左、右子集作为叶节点还是继续分裂 -split
    * 其一或都作为叶节点，get_leaf
    * 其一node继续分裂，get_best_split(left_X,left_y)
        * 重复3
        
将以上打包为train_tree函数

### 分裂出一个node：

In [20]:
def split_node(X, y, index, value):
    """ Split data set X, y based on a feature and a value
    Args:
        X, y (numpy.ndarray, data set)
        index (int, index of the feature used for splitting)
        value (value of the feature used for splitting)
    Returns:
        list, list (left and right child, a child is in the format of [X, y])
    """
    x_index = X[:, index]
    # 判定特征类型
    if type(X[0, index]) in [int, float]: # （第1行）第index维，如果是数值特征
        mask = x_index >= value  # mask = (x_index >= value)  布尔值检索
    else:
        mask = x_index == value # 类别特征
    # 分裂
    left = [X[~mask, :], y[~mask]]  # ~ 即是 !=，
    right = [X[mask, :], y[mask]]
    return left, right

### Define greedy search function:
试遍所有可能的splits，返回 weighted MSE最小的那个node

In [12]:
def get_best_split(X, y):
    """ Obtain the best splitting point and resulting children for data set X, y
    Args:
        X, y (numpy.ndarray, data set)
        criterion (gini or entropy)
    Returens:
        node ( dict {index: index of the features, values: feature value, 
                children: left and right childern} )
    """
    best_index, best_value, best_score, children = None, None, 1e10, None
    for index in range(len(X[0])): # 遍历全部特征，X第一行（向量）的长度即特征数
        for value in np.sort(np.unique(X[:, index])): # 遍历第index列的特征值
            groups = split_node(X, y, index, value) # 分裂成左、右2个子集 [[X,y],[X,y]]
            impurity = weighted_mse([groups[0][1], groups[1][1]]) 
            if impurity < best_score: # 阈值初始随便设置
                best_index, best_value, best_score, children = index, value, impurity, groups
    return {'index': best_index,'value':best_value,'children':children}

最终叶节点的值：

In [13]:
def get_leaf(targets):
    return np.mean(targets)

综上，递归分裂函数：

In [14]:
def split(node, max_depth, min_size, depth):
    """ Split children of a node to constant new nodes or assign them terminals
    Args:
        node (dict, with {'index':0,'value':'semi','children':([array[X,y], array[X,y]])})
        max_depth (int, maximal depth of the tree)
        min_size (int, minimal samples required to further split a child)
        depth (int, current depth of the node)
    """
    left, right = node['children'] #'children':([array[X,y], array[X,y]])
    del (node['children'])  # 删除字典中的children key及其value
    if left[1].size == 0:
        node['right'] = get_leaf(right[1]) 
        return             # node = {'index':0,'value':'semi','right':750}
    if right[1].size == 0:
        node['left'] = get_leaf(left[1])
        return
    if depth >= max_depth:
        node['left'],node['right'] = get_leaf(left[1]),get_leaf(right[1])
        return
    if left[1].size <= min_size: 
        node['left'] = get_leaf(left[1])
    else:  # 左边继续分裂
        result = get_best_split(left[0],left[1]) # 得到最佳node
        result_left, result_right = result['children'] # 确认左、右是否可为叶节点
        if result_left[1].size == 0:
            node['left'] = get_leaf(result_right[1])
        elif result_right[1].size ==0:
            node['left'] = get_leaf(result_left[1])
        else:
            node['left'] = result # 不为叶节点的，继续分裂，递归split
            split(node['left'], max_depth, min_size, depth +1)
    if right[1].size <= min_size:
        node['right'] = get_leaf(right[1])
    else: # 右边继续分裂
        result = get_best_split(right[0], right[1])
        result_left, result_right =  result['children']
        if result_left[1].size == 0:
            node['right'] = get_leaf(result_right[1])
        elif result_right[1].size == 0:
            node['right'] = get_leaf(result_left[1])
        else:
            node['right'] = result
            split(node['right'], max_depth, min_size, depth + 1)

### 训练：给定训练集，得到root，基于root 递归分裂：

In [15]:
def train_tree(X_train, y_train, max_depth, min_size):
    """ Construction of a tree starts here
    Args:
        X_train, y_train (list, list, trianing data)
        max_depth (int, maximal depth of the tree)
        min_size (int, minimal samples required to further split a child)
    """
    root = get_best_split(X_train, y_train)
    split(root, max_depth, min_size, 1)
    return root

### 测试之前手工计算的数据集：

In [18]:
X_train = np.array([['semi', 3],
                    ['detached', 2],
                    ['semi', 2],
                    ['detached',3],
                    ['semi', 4]], dtype=object)
y_train = np.array([600,700,400,800,700])

In [21]:
tree = train_tree(X_train, y_train, 2, 2)
tree

{'index': 0,
 'left': {'index': 1, 'left': 400.0, 'right': 650.0, 'value': 3},
 'right': 750.0,
 'value': 'detached'}

# 波士顿房价预测

In [23]:
from sklearn import datasets
boston = datasets.load_boston()

num_test = 10
X_train = boston.data[:-num_test,:]
y_train = boston.target[:-num_test]
X_test = boston.data[-num_test:,:]
y_test = boston.target[-num_test:]

In [25]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(max_depth=10, min_samples_split=3)

regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=3,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [27]:
predictions = regressor.predict(X_test)
print(predictions)
print(y_test)

[ 18.92727273  20.9         20.9         18.92727273  20.9         26.6
  20.73076923  30.25        28.2         20.73076923]
[ 19.7  18.3  21.2  17.5  16.8  22.4  20.6  23.9  22.   11.9]


回归森林

In [29]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=3)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [30]:
predictions = regressor.predict(X_test)
print(predictions)
print(y_test)

[ 18.8210883   20.90296531  21.51391577  20.39428754  21.00655679
  25.56203169  21.65525145  28.46614762  27.35573782  21.09353696]
[ 19.7  18.3  21.2  17.5  16.8  22.4  20.6  23.9  22.   11.9]
